In [4]:
#Importing the Necessary Packages
import re
import csv
import io  
from collections import Counter
from IPython.display import FileLink, display


FAILED_LOGIN_THRESHOLD = 10   # The default threshold for suspicious activity

def parse_log_file(file_path): #function for parsing the log file and returning relevant information
    ip_addresses = []
    endpoints = []
    failed_logins = []

    with open(file_path, 'r') as log_file:
        for line in log_file: 
            # For Extracting IP address from the log file
            ip_match = re.search(r'(\d+\.\d+\.\d+\.\d+)', line)
            if ip_match:
                ip_addresses.append(ip_match.group(1))

            # For Extracting endpoints from the log file
            endpoint_match = re.search(r'"GET (.*?) HTTP', line)
            if endpoint_match:
                endpoints.append(endpoint_match.group(1))

            #For Detecting failed login attempts
            if '401' in line or 'Invalid credentials' in line:
                if ip_match:
                    failed_logins.append(ip_match.group(1))

    return ip_addresses, endpoints, failed_logins

def count_requests_per_ip(ip_addresses): #For Counting requests per IP
    ip_count = Counter(ip_addresses)
    return ip_count.most_common()

def find_most_frequent_endpoint(endpoints): #For finding the most frequently accessed endpoint.
    endpoint_count = Counter(endpoints)
    return endpoint_count.most_common(1)[0]

def detect_suspicious_activity(failed_logins, threshold=FAILED_LOGIN_THRESHOLD):  #For detecting suspicious activity based on failed login attempts
    failed_login_count = Counter(failed_logins)
    return [(ip, count) for ip, count in failed_login_count.items()]

def save_results(ip_requests, most_accessed_endpoint, suspicious_ips): # For saving the results in a CSV file
    memory_file = io.StringIO()
    writer = csv.writer(memory_file)

    # Write Requests per IP
    writer.writerow(["IP Address", "Request Count"])
    writer.writerows(ip_requests)

    # Write Most Accessed Endpoint
    writer.writerow([])
    writer.writerow(["Most Accessed Endpoint"])
    writer.writerow(["Endpoint", "Access Count"])
    writer.writerow(most_accessed_endpoint)

    # Write Suspicious Activity
    writer.writerow([])
    writer.writerow(["Suspicious Activity"])
    writer.writerow(["IP Address", "Failed Login Count"])
    writer.writerows(suspicious_ips)

    memory_file.seek(0)
    return memory_file

def main():
    # Path to the log file
    log_file_path = r"C:\Users\ADMIN\Downloads\sample.log"  

    # Parse the log file
    ip_addresses, endpoints, failed_logins = parse_log_file(log_file_path)

    # Count requests per IP
    ip_requests = count_requests_per_ip(ip_addresses)
    print("IP Address           Request Count")
    for ip, count in ip_requests:
        print(f"{ip:<20}{count}")

    # Find the most frequently accessed endpoint
    most_accessed_endpoint = find_most_frequent_endpoint(endpoints)
    print("\nMost Frequently Accessed Endpoint:")
    print(f"{most_accessed_endpoint[0]} (Accessed {most_accessed_endpoint[1]} times)")

    # Detect suspicious activity
    suspicious_ips = detect_suspicious_activity(failed_logins)
    if suspicious_ips:
        print("\nSuspicious Activity Detected:")
        print("IP Address           Failed Login Attempts")
        for ip, count in suspicious_ips:
            print(f"{ip:<20}{count}")
    else:
        print("\nNo Suspicious Activity Detected.")

    #Saving the results
    memory_file = save_results(ip_requests, most_accessed_endpoint, suspicious_ips)

    
    output_path = "log_analysis_results.csv"
    with open(output_path, "w", newline='',encoding='utf-8') as f:
        f.write(memory_file.getvalue())
    # Downloadable file link
    print(f"\nResults saved to: {output_path}")
    display(FileLink(output_path))

if __name__ == "__main__":
    main()


IP Address           Request Count
203.0.113.5         8
198.51.100.23       8
192.168.1.1         7
10.0.0.2            6
192.168.1.100       5

Most Frequently Accessed Endpoint:
/home (Accessed 5 times)

Suspicious Activity Detected:
IP Address           Failed Login Attempts
203.0.113.5         8
192.168.1.100       5

Results saved to: log_analysis_results.csv


C:\Users\ADMIN\log_analysis_results.csv